In [447]:
import pandas as pd

df = pd.read_csv("data/Seasons_Stats.csv")

df.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [448]:
dropped_columns = ["Pos", "Age", "Tm"]

df.drop(dropped_columns,  axis=1, inplace=True)
df = df.fillna(0)
df.head()

,Unnamed: 0,Year,Player,G,GS,MP,PER,TS%,3PAr,FTr,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,63.0,0.0,0.0,0.0,0.368,0.0,0.467,...,0.705,0.0,0.0,0.0,176.0,0.0,0.0,0.0,217.0,458.0
1,1,1950.0,Cliff Barker,49.0,0.0,0.0,0.0,0.435,0.0,0.387,...,0.708,0.0,0.0,0.0,109.0,0.0,0.0,0.0,99.0,279.0
2,2,1950.0,Leo Barnhorst,67.0,0.0,0.0,0.0,0.394,0.0,0.259,...,0.698,0.0,0.0,0.0,140.0,0.0,0.0,0.0,192.0,438.0
3,3,1950.0,Ed Bartels,15.0,0.0,0.0,0.0,0.312,0.0,0.395,...,0.559,0.0,0.0,0.0,20.0,0.0,0.0,0.0,29.0,63.0
4,4,1950.0,Ed Bartels,13.0,0.0,0.0,0.0,0.308,0.0,0.378,...,0.548,0.0,0.0,0.0,20.0,0.0,0.0,0.0,27.0,59.0


In [449]:
grouped = df.groupby(['Player'])

grouped = list(grouped)

type(grouped[0])

tuple

In [450]:
def padding_(seasons, seq_len):
    print(len(seasons))
    new_seasons = []
    for season in seasons:
        print(season.shape[0])
        new_seasons.append(np.pad(season, [(0, seq_len - season.shape[0]), (0,0)], mode='constant'))
    new_seasons = np.array(new_seasons)
    print(new_seasons.shape)
    print(new_seasons[0].shape, new_seasons[1].shape)
    return new_seasons

In [451]:
import numpy as np
goat_tier = ["Michael Jordan*", "LeBron James", "Kareem Abdul-Jabbar*", "Kobe Bryant", "Wilt Chamberlain*", "Magic Johnson*"]

X = []
y = []
seq_len = 22

for i in range(len(grouped)):
    
    player, player_df = grouped[i]
    df_np = player_df.to_numpy()
    df_np = np.delete(df_np, [2], 1)[:seq_len]
    X.append(df_np)
    if player in goat_tier:
        y.append(1)
    else:
        y.append(0)

X = X[1:]
y = y[1:]
y = np.array([y])
X = padding_(X, seq_len)
        


3921
18
1
2
3
1
9
1
1
15
3
11
2
5
17
1
1
20
2
6
9
3
9
5
14
2
3
5
4
8
19
11
12
11
10
6
1
4
4
6
1
3
2
20
2
10
1
13
4
1
2
8
8
8
8
7
9
8
2
12
5
1
2
5
9
6
4
1
2
1
4
1
1
17
2
2
8
1
4
1
1
3
6
2
2
6
9
4
1
9
12
1
4
20
10
1
1
3
13
17
2
3
18
13
1
1
12
8
1
11
4
16
12
15
2
9
8
3
1
5
2
13
3
22
3
2
4
6
8
4
10
1
13
2
10
14
10
12
2
3
1
14
2
7
8
3
10
2
4
1
1
9
1
2
1
16
4
18
8
4
6
8
4
15
6
5
3
16
1
21
7
13
11
13
9
15
1
8
5
5
1
13
1
18
1
14
8
1
2
15
17
10
8
17
1
12
4
6
2
5
17
4
12
2
2
4
9
5
12
4
1
6
5
5
1
6
7
14
1
1
7
2
1
3
12
14
12
7
7
22
3
4
15
1
12
12
17
11
3
1
1
1
1
1
5
1
2
5
1
1
7
4
1
11
2
1
1
4
4
12
4
18
7
1
19
21
3
14
5
5
1
2
1
2
1
8
1
10
19
1
8
15
1
3
9
4
1
1
6
4
10
3
1
8
4
1
16
1
3
1
7
2
1
1
9
13
11
1
1
2
3
10
10
13
10
1
9
1
4
6
13
9
14
13
6
1
5
6
18
13
4
6
8
1
1
8
3
7
12
3
1
3
1
17
1
1
1
1
6
13
5
1
2
1
4
4
4
14
13
6
1
1
4
6
3
1
1
10
2
8
11
7
11
1
4
7
7
1
1
16
6
1
16
2
20
5
4
1
1
4
5
11
4
1
6
1
1
12
1
4
3
12
10
3
1
6
14
1
7
2
1
1
2
7
2
1
7
14
17
4
9
2
10
2
8
2
1
4
5
5
1
14
2
18
1
10
2
2
8
19
5
17

In [452]:
y

array([[0, 0, 0, ..., 0, 0, 0]])

In [453]:
import torch
import torch.nn as nn
device = "cuda:0"
class GOATnn(nn.Module):
    def __init__(self,  hidden_dim, n_layers, feature_size = 49, seq_len = 22, drop_prob=0.5):
        super(GOATnn, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(feature_size , hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [454]:
import torch
import torch.nn as nn
device = "cuda:0"
class GOATnn(nn.Module):

    def __init__(self, batch_size, seq_len = 22, feature_size = 49, drop_prob = 0.35):
        super(GOATnn, self).__init__()
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.feature_size = feature_size
        
        self.linear1 = nn.Linear( feature_size * seq_len, 128)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(128, 32)
        self.dropout = nn.Dropout(0.3)
        self.linear3 = nn.Linear(32,4)
        self.linear4 = nn.Linear(4, 2)

        self.sig = nn.Softmax()
    def forward(self, input):
        input = input.reshape(self.batch_size, self.feature_size * self.seq_len)
        input = self.linear1(input)
        input = self.relu(input)
        input = self.linear2(input)
        input = self.linear3(input)
        
        input = self.relu(input)
        input = self.linear4(input)
        return input
   

In [455]:
batch_size = 1307

model = GOATnn(batch_size)
model.to(device)

GOATnn(
  (linear1): Linear(in_features=1078, out_features=128, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=128, out_features=32, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (linear3): Linear(in_features=32, out_features=4, bias=True)
  (linear4): Linear(in_features=4, out_features=2, bias=True)
  (sig): Softmax(dim=None)
)

In [456]:
lr=1e-9
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [457]:
batch_size = 1307

In [458]:
from torch.utils.data import TensorDataset, DataLoader
batch_inputs = X.astype(np.float32)

batch_outputs = y.T
print(torch.from_numpy(batch_inputs).shape, torch.from_numpy(batch_outputs).shape)
train_data = TensorDataset(torch.from_numpy(batch_inputs), torch.from_numpy(batch_outputs))

# dataloaders

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)


torch.Size([3921, 22, 49]) torch.Size([3921, 1])


In [460]:
epochs = 10000
counter = 0
print_every = 500
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    
    for inputs, labels in train_loader:
        counter += 1
        inputs, labels = inputs.to(device), labels.to(device)
        output = model(inputs)
        loss = criterion(output.squeeze(), labels.squeeze())
        optimizer.zero_grad()

        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()))

Epoch: 167/10000... Step: 500... Loss: 0.007130...
Epoch: 334/10000... Step: 1000... Loss: 0.006494...
Epoch: 500/10000... Step: 1500... Loss: 0.005989...
Epoch: 667/10000... Step: 2000... Loss: 0.007141...
Epoch: 834/10000... Step: 2500... Loss: 0.003836...
Epoch: 1000/10000... Step: 3000... Loss: 0.002764...
Epoch: 1167/10000... Step: 3500... Loss: 0.004984...
Epoch: 1334/10000... Step: 4000... Loss: 0.006487...
Epoch: 1500/10000... Step: 4500... Loss: 0.003322...
Epoch: 1667/10000... Step: 5000... Loss: 0.004845...
Epoch: 1834/10000... Step: 5500... Loss: 0.005542...
Epoch: 2000/10000... Step: 6000... Loss: 0.004054...
Epoch: 2167/10000... Step: 6500... Loss: 0.001855...
Epoch: 2334/10000... Step: 7000... Loss: 0.002114...
Epoch: 2500/10000... Step: 7500... Loss: 0.001698...
Epoch: 2667/10000... Step: 8000... Loss: 0.004394...
Epoch: 2834/10000... Step: 8500... Loss: 0.002216...
Epoch: 3000/10000... Step: 9000... Loss: 0.001725...
Epoch: 3167/10000... Step: 9500... Loss: 0.002442...

In [461]:
players = []
for i in range(len(grouped)):
    player, _ = grouped[i]
    players.append(player)
players = players[1:]


In [ ]:
goat_tier

['Michael Jordan*',
 'LeBron James',
 'Kareem Abdul-Jabbar*',
 'Kobe Bryant',
 'Wilt Chamberlain*',
 'Magic Johnson*']

In [ ]:
index = players.index("Kim Hughes")
index

2308

In [ ]:
mj = batch_inputs[index]
print(mj)

[[4802. 1977.   81. ...    0.  308.  321.]
 [5169. 1978.   56. ...   57.  163.  123.]
 [5533. 1979.   81. ...   78.  215.  214.]
 ...
 [   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]]


In [478]:
batch_size = 1
model.batch_size = 1
player_score = []

for batch_input in batch_inputs:
    probs = model(torch.from_numpy(batch_input).cuda())
    player_score.append(torch.nn.functional.softmax(probs)[0][1].item())

<ipython-input-478-68a05aae08ec>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  player_score.append(torch.nn.functional.softmax(probs)[0][1].item())


In [ ]:
players

['A.C. Green',
 'A.J. Bramlett',
 'A.J. English',
 'A.J. Guyton',
 'A.J. Hammons',
 'A.J. Price',
 'A.J. Wynder',
 'A.W. Holt',
 'Aaron Brooks',
 'Aaron Gordon',
 'Aaron Gray',
 'Aaron Harrison',
 'Aaron James',
 'Aaron McKie',
 'Aaron Miles',
 'Aaron Swinson',
 'Aaron Williams',
 'Abdul Jeelani',
 'Acie Earl',
 'Acie Law',
 'Adam Harrington',
 'Adam Keefe',
 'Adam Morrison',
 'Adonal Foyle',
 'Adonis Jordan',
 'Adonis Thomas',
 'Adreian Payne',
 'Adrian Branch',
 'Adrian Caldwell',
 'Adrian Dantley*',
 'Adrian Griffin',
 'Adrian Smith',
 'Al Attles',
 'Al Bianchi',
 'Al Butler',
 'Al Carlson',
 'Al Cervi*',
 'Al Eberhard',
 'Al Ferrari',
 'Al Fleming',
 'Al Guokas',
 'Al Hairston',
 'Al Harrington',
 'Al Henry',
 'Al Horford',
 'Al Jackson',
 'Al Jefferson',
 'Al Masino',
 'Al Miksis',
 'Al Roges',
 'Al Skinner',
 'Al Thornton',
 'Al Tucker',
 'Al Wood',
 'Al-Farouq Aminu',
 'Alaa Abdelnaby',
 'Alan Anderson',
 'Alan Hardy',
 'Alan Henderson',
 'Alan Ogg',
 'Alan Sawyer',
 'Alan Willi

In [486]:
list1, list2 = (list(x) for x in zip(*sorted(zip(player_score, players), reverse=True,key=lambda pair: pair[0])))

In [487]:
for i in range(len(list1)):
    print(list1[i], list2[i])

0.9999934434890747 Wilt Chamberlain*
0.5859144330024719 Kareem Abdul-Jabbar*
0.5859144330024719 Kobe Bryant
0.5854150652885437 Michael Jordan*
0.5716152191162109 Magic Johnson*
0.562740683555603 Tiny Archibald*
0.4863567650318146 Kevin Johnson
0.4129873812198639 David Robinson*
0.37233981490135193 Larry Smith
0.3417116403579712 LeBron James
0.251480370759964 Ennis Whatley
0.0015960573218762875 Mike Gminski
0.001526398817077279 Dana Barros
0.0013974832836538553 Mike Bantom
0.0010087293339893222 Dwight Jones
0.0009004998719319701 Reggie Theus
0.0006059217848815024 Rick Fox
0.00018873426597565413 Dave Corzine
2.0087869415874593e-05 Otto Moore
1.9310447896714322e-05 Artis Gilmore*
1.7586749891052023e-05 Wes Unseld*
1.651544516789727e-05 Kent Benson
1.4963289686420467e-05 Johnny Egan
1.4043216651771218e-05 Terry Teagle
8.227051694120746e-06 Chuck Person
5.061855972599005e-06 Gar Heard
4.9094287533080205e-06 John Tresvant
3.789579750446137e-06 Walt Wesley
2.970832156279357e-06 Dillard Crocke